In [59]:
import xarray as xr
import pandas as pd
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as pl
import cartopy.crs as ccrs
from cartopy import feature as cfeature

import matplotlib.ticker as mticker
import matplotlib.dates as mdates

In [60]:
def plot_on_map(lon, lat, arr, cmap, LON0, vmin=None, vmax=None, label="SST"):
    """
    Plots given array using pcolormesh on a map using Plate Carree projection.
    """
    lats, lons = np.meshgrid(lat, lon)

    # set up figure
    LON0 = 180.
    TIKLABFS = 10
    AXLABFS = 12
    map_proj = ccrs.PlateCarree(central_longitude=LON0)
    fig = pl.figure(figsize=[8, 6])
    ax = fig.add_axes([0.10, 0.10, 0.90, 0.80], projection=map_proj)

    # plot array using pcolormesh
    im = ax.pcolormesh(lons, lats, arr, cmap=cmap,
                       vmin=vmin, vmax=vmax,
                       rasterized=True, alpha=0.95,
                       transform=ccrs.PlateCarree()
                       )
    cb = pl.colorbar(im, shrink=0.90)
    cb.set_label("%s (deg C)" % label, fontsize=AXLABFS)

    # add land colors and coastlines
    land_50m = cfeature.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='face',
                                        facecolor="lightgray",
                                        alpha=0.65, zorder=-1)
    ax.add_feature(land_50m)
    ax.coastlines(linewidth=0.50, color="k", alpha=0.50)

    # set limits of the plot and grid lines
    ax.set_ylim(-66, 66)
    ax.set_xlim(-60., 120)
    gl = ax.gridlines(draw_labels=True, dms=True,
                      x_inline=False, y_inline=False,
                      linestyle="-", linewidth=0.50,
                      color="0.5",alpha=0.15)

    # add axis ticks and set tick label sizes
    gl.xlocator = mticker.FixedLocator(np.arange(-180, 181, 20))
    gl.ylocator = mticker.FixedLocator(np.arange(-65, 66, 15))
    gl.xlabel_style = {"fontsize": TIKLABFS}
    gl.ylabel_style = {"fontsize": TIKLABFS}

    return fig